In [21]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def fonction_scraping():
    url = input("Collez l'URL wikipedia :  ")
    page = requests.get(url)

    if page.status_code == 200:
        print("Url valide")
        contenue = BeautifulSoup(page.content, "html.parser")
        TitreP = contenue.find("h1")
        Titretext = TitreP.text
        print(f"L'URL saisie a pour titre principal : {Titretext}")

        Ensemble = contenue.find_all(["h1", "h2", "h3", "h4", "h5", "h6", "p"])
        EnsenDico = []
        Dico = None

        for ensemble in Ensemble:
            if ensemble.name.startswith("h"):
                Dico = {
                    "Titre": ensemble.get_text(strip=True),
                    "type": ensemble.name,
                    "paragraphe": []
                }
                EnsenDico.append(Dico)
            elif ensemble.name == "p" and Dico is not None:
                Dico["paragraphe"].append(ensemble.get_text(strip=True))

        # Création d’un DataFrame avec titres et paragraphes
        data_struct = []
        for bloc in EnsenDico:
            data_struct.append({
                "Titre": bloc["Titre"],
                "Type": bloc["type"],
                "Paragraphe": " ".join(bloc["paragraphe"])
            })
        df_contenu = pd.DataFrame(data_struct)

        # Scraping des liens internes
        Ensemble_lien = contenue.find_all('a', href=re.compile("^/wiki"))
        print("Nous avons ici l'ensemble des liens de redirection :")
        Liens = ["https://fr.wikipedia.org" + lien["href"] for lien in Ensemble_lien]

        df_liens = pd.DataFrame(Liens, columns=["Lien_wikipedia"])

        return df_contenu, df_liens

    else:
        print("URL non valide")
        return pd.DataFrame(), pd.DataFrame()

# Lancement
df_contenu, df_liens = fonction_scraping()

print("\n📘 DataFrame contenu :")
print(df_contenu.head())

print("\n🔗 DataFrame liens :")
print(df_liens.head())


Collez l'URL wikipedia :   https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Accueil_principal


Url valide
L'URL saisie a pour titre principal : Bienvenue sur Wikipédia
Nous avons ici l'ensemble des liens de redirection :

📘 DataFrame contenu :
                       Titre Type  \
0    Bienvenue sur Wikipédia   h1   
1                  Wikipédia   h2   
2  Article labellisé du jour   h2   
3                 Actualités   h2   
4           Le saviez-vous ?   h2   

                                          Paragraphe  
0                                                     
1     L'encyclopédie libre que vous pouvez améliorer  
2  Lavallée des Rouetsest le nom donné à une part...  
3                                                     
4                                                     

🔗 DataFrame liens :
                                      Lien_wikipedia
0  https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:A...
1      https://fr.wikipedia.org/wiki/Portail:Accueil
2  https://fr.wikipedia.org/wiki/Sp%C3%A9cial:Pag...
3  https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:C...
4    https:/